In [ ]:
# src/experiments.py
import time
import numpy as np
import pandas as pd
from .neural_network import NeuralNetwork
from .softmax_regression import SoftmaxRegression
from .utils import evaluate, save_performance

def run_softmax_grid(X_train, y_train, X_val, y_val, lrs=(0.1,0.2,0.3), epochs=8, batch=512, wd=1e-4):
    rows = []
    for lr in lrs:
        sm = SoftmaxRegression(input_dim=X_train.shape[1], n_classes=3)
        t0 = time.time()
        _ = sm.fit(X_train, y_train, lr=lr, epochs=epochs, batch_size=batch,
                   X_val=X_val, y_val=y_val, weight_decay=wd, patience=2, verbose=False)
        secs = time.time() - t0
        yv = sm.predict(X_val)
        m = evaluate(y_val, yv)
        rows.append({
            "model":"softmax", "arch":"linear", "activation":"softmax",
            "lr":lr, "epochs":epochs, "batch_size":batch, "wd":wd,
            "val_acc":m["acc"], "val_f1":m["f1_macro"], "train_time_s":round(secs,1)
        })
    return pd.DataFrame(rows)

def run_nn_grid(X_train, y_train, X_val, y_val,
                archs=((256,128),(512,256,128)), activations=("relu","tanh"),
                lrs=(0.03,0.02), epochs=20, batch=256, wd=1e-4):
    rows = []
    for arch in archs:
        for act in activations:
            for lr in lrs:
                layers = [X_train.shape[1], *arch, 3]
                nn = NeuralNetwork(layers=layers, activation=act)
                t0 = time.time()
                _ = nn.fit(X_train, y_train, lr=lr, epochs=epochs, batch_size=batch,
                           X_val=X_val, y_val=y_val, weight_decay=wd, patience=3, verbose=False)
                secs = time.time() - t0
                yv = nn.predict(X_val)
                m = evaluate(y_val, yv)
                rows.append({
                    "model":"nn", "arch":str(list(arch)), "activation":act,
                    "lr":lr, "epochs":epochs, "batch_size":batch, "wd":wd,
                    "val_acc":m["acc"], "val_f1":m["f1_macro"], "train_time_s":round(secs,1)
                })
    return pd.DataFrame(rows)

def save_grid(df, path_csv):
    return save_performance(df.to_dict(orient="records"), path_csv)
